### 数据读入

In [ ]:
import numpy
from DataSet import TestDataset
from DataSet import TrainDataset
from torch.utils.data import Dataset
from NET import NET_1D_CNN
from NET import NET_1D_CNN_NEW
from NET import NET_1D_CNN_16C
import numpy as np
import torch.nn as nn
import torch
import time
import matplotlib.pyplot as plt

use_gpu = torch.cuda.is_available()

STARTTIME = time.time()

train = TrainDataset()
train.__int__()
test = TestDataset()
test.__int__()


each_size = 10
test_size=1

train_dataloader = torch.utils.data.DataLoader(train, batch_size=each_size, shuffle=True)#,num_workers=16多线程，会变慢？？    #, batch_size=each_size
test_dataloader = torch.utils.data.DataLoader(test, batch_size=test_size, shuffle=True)#,num_workers=16

In [13]:
#输出长度
print(len(train_dataloader))
print(len(test_dataloader))


240
600


### 训练部分

In [7]:

NET_1D_CNN_16C= nn.Sequential(
                    nn.Conv1d(16,9,3),
                    nn.Conv1d(9,5,3),
                    nn.Conv1d(5,1,5),
                    #nn.MaxPool1d(2,2),
                    nn.Flatten()
                    nn.Linear(512, 320),
                    nn.ELU(),
                    nn.Linear(320, 128),
                    nn.ELU(),
                    nn.Linear(128, 20),
                    )


net = NET_1D_CNN_16C
print(net)

loss_fn = nn.CrossEntropyLoss()


learning_rate = 0.005
epoch = 50

optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate)

#初始化网络参数
def init_weights(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, std=0.1)
    elif type(m) == nn.Conv1d:
        nn.init.normal_(m.weight, std=0.1)
net.apply(init_weights)                     

T1TIME = time.time()

sunshi=[]
zhunquelv=[]
nsunshi=[]

for i in range(epoch):
    
    a = 0
    ave = 0
    if use_gpu:
        net.cuda()
    net.train()

    for data in train_dataloader:
        train_data, train_label = data
        outputs_label = net(train_data.cuda())
        outputs_label = outputs_label.reshape(1, each_size, 20)
        matrix = [[0 for j in range(20)] for h in range(each_size)]  # 把标签处理为01矩阵
        for k in range(each_size):
            matrix[k][train_label[k]] = 1
        matrix = torch.from_numpy(np.array(matrix).reshape(1, each_size, 20)).to(torch.float32).cuda()
        loss = loss_fn(outputs_label, matrix)
        # print("train_loss:", loss)
        a += 1
        ave = ave + loss
        nave=ave.cpu().tolist()

        optimizer.zero_grad()  # 梯度清零
        loss.backward()  # 误差反向传播
        optimizer.step()  # 参数更新
    
    
    #准确率计算
    net.eval()
    with torch.no_grad():
        test=[]
        true=[]
        right=0
        for tdata in test_dataloader:
            
            test_data, test_label = tdata
            test_outputs = net(test_data.cuda()).cpu().numpy()
            where=np.where(test_outputs ==np.max(test_outputs))
            test.append(where[2])
            true.append(test_label)
        for ij in range(len(test)):
            if (int(test[ij])==int(true[ij])):
                right=right+1
    zhunquelv.append(right/len(test))
    sunshi.append(ave/a)
    nsunshi.append(nave/a)
    
    print('epoch:',i,"准确率为:", right/len(test),'平均损失:',nave / a)
    # print("准确率为:", right/len(test))
    # print('平均损失:',nave / a)#打印每一轮的平均损失

print('准确率',zhunquelv)
print('损失',nsunshi)

epochs = range(0,epoch)
plt.figure()
plt.subplot(1,2,1)
plt.plot(epochs,zhunquelv,c='green', label="准确率")
plt.subplot(1,2,2)
plt.plot(epochs,nsunshi,c='blue', label="平均损失")
plt.show()


T2TIME = time.time()

torch.save(net, 'CNN.model')            #保存模型
ENDTIME = time.time()

print("训练时间：",T2TIME - T1TIME)
print(ENDTIME - STARTTIME)              #输出耗时

SyntaxError: invalid syntax (2597467026.py, line 7)

### 混淆矩阵

In [ ]:
#混淆矩阵模块
net.eval()
with torch.no_grad():
    ntest=[]
    ntrue=[]
    nright=0
    for tdata in test_dataloader:
        test_data, test_label = tdata
        test_outputs = net(test_data.cuda()).cpu().numpy()
        where=np.where(test_outputs ==np.max(test_outputs))
        ntest.append(where[2])
        ntrue.append(test_label)

    print(len(ntest))
    cm=np.zeros((20,20), dtype = int)                       #混淆矩阵初始化
    for i in range(0,len(ntest)):
        cm[int(ntest[i])][int(ntrue[i])]=cm[int(ntest[i])][int(ntrue[i])]+1

    for i in range(0,len(ntest)):
        if (int(ntest[i])==int(ntrue[i])):
            nright=nright+1
    print(cm)

    print("准确率为：", nright/len(ntest))


## 代码测试部分

### 测试混淆矩阵

In [ ]:
import numpy as np
test=np.zeros((20,20),dtype = int)
testout=np.array([3,3,3])
tureout=np.array([1,2,3])
print (len(testout))
for i in range(0,len(testout)):
    test[testout[i]][tureout[i]]=test[testout[i]][tureout[i]]+1
test[1][3]=1
print (test)
print (test.shape)

In [ ]:
# coding=utf-8
import matplotlib.pyplot as plt

x=[0,0.5,0.8,1.0,1.2,1.4,1.6,1.8,2.0,2.2,2.4,3,4,5,6]
y=[99.9,99.9,99.7,94.4,66.5,32.2,11.7,4.4,2.9,3.1,3.5,3.7,3.1,2.6,2.3]
plt.figure(1)
plt.plot(x,y)
plt.show()
# epoch = range(1,11)
# plt.figure(1)
# plt.plot(epoch,zhunquelv,c='green', label="准确率")
# plt.figure(2)
# plt.plot(epoch,nsunshi,c='blue', label="平均损失")

plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

x = np.linspace(1, 2, 2)
y1 = np.sin(x)

y2 = np.cos(x)

plt.subplot(1, 2, 1, frameon = False) # 两行一列，位置是1的子图
plt.plot(x, y1, 'b--')
plt.ylabel('y1')
plt.subplot(1, 2, 2, projection = 'polar')
plt.plot(x, y2, 'r--')

# plt.subplot(2, 2, 3, sharex = ax1, facecolor = 'red')
# plt.plot(x, y2, 'r--')
# plt.ylabel('y2')

plt.show()

